# Load Packages

In [51]:
import os
import cv2
import copy

import math
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from glob import glob
import tqdm
from skimage.metrics import structural_similarity as compare_ssim
import shutil
#Parsing/Modifying XML
from lxml.etree import Element,SubElement,tostring
import xml.dom.minidom
from xml.dom.minidom import parseString
import xml.etree.ElementTree as et
from xml.dom import minidom

import data_eng.az_proc as ap
import data_eng.form_calcs as fc


# File Paths

In [2]:
parent_directory = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//"
tile_names_tile_urls_complete_array = np.load("image_download_azure/tile_name_tile_url_complete_array.npy")
tile_dir_path = os.path.join(parent_directory,"complete_dataset","tiles")
#https://stackoverflow.com/questions/5953373/how-to-split-image-into-multiple-pieces-in-python
img_anno_directory = os.path.join(parent_directory,"temp/rechip")
remaining_chips_path =  os.path.join(parent_directory,"temp/remaining_chips")
remaining_sets_path =  os.path.join(parent_directory,"temp/remaining_sets")

verified_standard_quad_image_name_format = os.path.join(parent_directory,"verified/verified_standard_quad_image_name_format")
incorrectly_chipped_images_path = os.path.join(parent_directory,"temp/incorrectly_chipped_images")
incorrect_named_correctly_chipped_dir = os.path.join(parent_directory,"temp/incorrectly_named_correct_chipped_images")

In [22]:
compiled_by_tile_dir = os.path.join(parent_directory,"compiled_dataset/compile_by_tile")
images_do_not_match_names_dir = os.path.join(parent_directory,"compiled_dataset/images_do_not_match_names_state_year")
correctly_chipped_incorrect_dir = os.path.join(parent_directory,"compiled_dataset/correctly_chipped_incorrect_data")

# Download Additional Tiles 

If necessary, download additional tiles
Note: Reform into function, to call above.

## Set 3

In [35]:
unverified_set3_subfolders_path = os.path.join(parent_directory,"unverified_images\student_reviewed_unverified_images_set3")
unverified_set3_subfolders_paths = ap.img_path_anno_path(ap.list_of_sub_directories(unverified_set3_subfolders_path))


## Get a list of the tiles in the subfolder
tiles = []
for directory in tqdm.tqdm(unverified_set3_subfolders_paths):
    #get all the image and xml paths in directory of annotated images
    #print(directory)
    fc.remove_thumbs(directory[0])
    image_paths = glob(directory[0] + "/*.jpg", recursive = True)
    xml_paths = glob(directory[1] + "/*.xml", recursive = True)
    #print(len(image_paths),len(xml_paths))

    #identify tiles in each folder
    for image in image_paths:
        image_name = os.path.splitext(os.path.basename(image))[0]
        tile_name = image_name.split("_",4)[-1].rsplit("_",1)[0]
        tiles.append(tile_name)
tiles = np.unique(tiles)

## Get a list of the tiles downloaded (without the extension)
tiles_downloaded =  os.listdir(tile_dir_path)
tiles_downloaded_wo_ext = []
for tile_downloaded in tiles_downloaded:
    tiles_downloaded_wo_ext.append(os.path.splitext(tile_downloaded)[0])
print(len(tiles_downloaded_wo_ext))

## Get a list of the the tiles that need to be downloaded
tiles_to_download = list(set(tiles.tolist()) - set(tiles_downloaded_wo_ext)) 

## Download Tiles
tile_names = []
tile_urls = []
file_names = []
for tile in tiles_to_download:   
    ### download the tiles if they are not in the tiles folder 
    #check if the tile name is contained in the string of complete arrays
    tile_name = [string for string in tile_names_tile_urls_complete_array[:,0] if tile in string]          
    if len(tile_name) == 1: #A single tile name # get tile url from the first (only) entry
        tile_url = tile_names_tile_urls_complete_array[tile_names_tile_urls_complete_array[:,0]==tile_name[0]][0][1] 
        tile_names.append(tile_name[0])
        tile_urls.append(tile_url)
    elif len(np.unique(tile_name)) > 1: # Multiple (different tiles) possibly the same tiles in different states, possible different years
        tile_url = tile_names_tile_urls_complete_array[tile_names_tile_urls_complete_array[:,0]==tile_name[0]][0][1]# get tile url
        tile_names.append(tile_name[0])
        tile_urls.append(tile_url)
    elif (len(tile_name) > 1): #Multiple different tile names that are the same, probably different naip storage locations
        # get tile url from the second entry 
        tile_url = tile_names_tile_urls_complete_array[tile_names_tile_urls_complete_array[:,0]==tile_name[1]][1][1] 
        tile_names.append(tile_name[1])
        tile_urls.append(tile_url)

    #get file name
    file_name = tile_name[0]
    if tile_name[0].count("_") > 5:
        tile_name = tile_name[0].rsplit("_",1)[0]
        file_name = tile_name + ".tif"
    print(file_name)
    ### Download tile
    file_names.append(ap.download_url(tile_url, tiles_complete_dataset_path,
                                                 destination_filename = file_name,       
                                                         progress_updater=ap.DownloadProgressBar()))

m_3411762_sw_11_060_20180822.tif


100% (408585536 of 408585536) |##########| Elapsed Time: 0:00:42 Time:  0:00:42


...done, 408585536 bytes.
m_3009612_nw_14_060_20181102.tif


100% (438599822 of 438599822) |##########| Elapsed Time: 0:00:43 Time:  0:00:43


...done, 438599822 bytes.
m_3809031_nw_15_060_20181121.tif


...

...done, 508105636 bytes.
m_3508057_ne_17_060_20181107.tif


100% (445281987 of 445281987) |##########| Elapsed Time: 0:00:41 Time:  0:00:41


...done, 445281987 bytes.
m_3909451_se_15_060_20180923.tif


100% (421702554 of 421702554) |##########| Elapsed Time: 0:00:36 Time:  0:00:36


...done, 421702554 bytes.


## Set 4

In [42]:
## Get a list of the tiles in the subfolder
unverified_set4_subfolders_path = os.path.join(parent_directory,"unverified_images\student_reviewed_unverified_images_set4")
unverified_set4_subfolders_paths = ap.img_path_anno_path(ap.list_of_sub_directories(unverified_set4_subfolders_path))


tiles = []
for directory in tqdm.tqdm(unverified_set4_subfolders_paths):
    #get all the image and xml paths in directory of annotated images
    #print(directory)
    fc.remove_thumbs(directory[0])
    image_paths = glob(directory[0] + "/*.jpg", recursive = True)
    xml_paths = glob(directory[1] + "/*.xml", recursive = True)
    #print(len(image_paths),len(xml_paths))

    #identify tiles in each folder
    for image in image_paths:
        image_name = os.path.splitext(os.path.basename(image))[0]
        tile_name = image_name.split("_",4)[-1].rsplit("_",1)[0]
        tiles.append(tile_name)
tiles = np.unique(tiles)

## Get a list of the tiles downloaded (without the extension)
tiles_downloaded =  os.listdir(tiles_complete_dataset_path)
tiles_downloaded_wo_ext = []
for tile_downloaded in tiles_downloaded:
    tiles_downloaded_wo_ext.append(os.path.splitext(tile_downloaded)[0])
print("number of tiles downloaded", len(tiles_downloaded_wo_ext))

## Get a list of the the tiles that need to be downloaded
tiles_to_download = list(set(tiles.tolist()) - set(tiles_downloaded_wo_ext)) 

## Download Tiles
tile_names = []
tile_urls = []
file_names = []
print("number of tiles that need to be downloaded", len(tiles_to_download))
for tile in tiles_to_download:   
    ### download the tiles if they are not in the tiles folder 
    #check if the tile name is contained in the string of complete arrays
    tile_name = [string for string in tile_names_tile_urls_complete_array[:,0] if tile in string]          
    if len(tile_name) == 1: #A single tile name # get tile url from the first (only) entry
        tile_url = tile_names_tile_urls_complete_array[tile_names_tile_urls_complete_array[:,0]==tile_name[0]][0][1] 
        tile_names.append(tile_name[0])
        tile_urls.append(tile_url)
    elif len(np.unique(tile_name)) > 1: # Multiple (different tiles) possibly the same tiles in different states, possible different years
        tile_url = tile_names_tile_urls_complete_array[tile_names_tile_urls_complete_array[:,0]==tile_name[0]][0][1]# get tile url
        tile_names.append(tile_name[0])
        tile_urls.append(tile_url)
    elif (len(tile_name) > 1): #Multiple different tile names that are the same, probably different naip storage locations
        # get tile url from the second entry 
        tile_url = tile_names_tile_urls_complete_array[tile_names_tile_urls_complete_array[:,0]==tile_name[1]][1][1] 
        tile_names.append(tile_name[1])
        tile_urls.append(tile_url)

    #get file name
    file_name = tile_name[0]
    if tile_name[0].count("_") > 5:
        tile_name = tile_name[0].rsplit("_",1)[0]
        file_name = tile_name + ".tif"
    print(file_name)
    ### Download tile
    file_names.append(ap.download_url(tile_url, tiles_complete_dataset_path,
                                                 destination_filename = file_name,       
                                                         progress_updater=ap.DownloadProgressBar()))

100%|██████████| 14/14 [00:00<00:00, 36.00it/s]


2021
118
m_4208355_sw_17_060_20180707.tif


100% (512753778 of 512753778) |##########| Elapsed Time: 0:00:55 Time:  0:00:55


...done, 512753778 bytes.
m_3110238_sw_13_060_20180924.tif


100% (495216713 of 495216713) |##########| Elapsed Time: 0:00:42 Time:  0:00:42


...

...done, 381201842 bytes.
m_4408523_nw_16_060_20180912.tif


100% (513328615 of 513328615) |##########| Elapsed Time: 0:01:07 Time:  0:01:07


...done, 513328615 bytes.
m_3908004_ne_17_060_20180701.tif


100% (448611597 of 448611597) |##########| Elapsed Time: 0:01:09 Time:  0:01:09


...done, 448611597 bytes.


# Images with state/year naming format
- Confirm that the name correctly matches the content (the name and tile/position match_
- Identify remaining images
- Rename labeled images/xmls 

## Unverified sets

### Set 1

# TESTING

In [3]:
def get_unique_tile_names(paths):
    tile_names = []
    file_names = []
    for image in paths:
        image_name = os.path.splitext(os.path.basename(image))[0]
        if image_name.count("_") > 9:
            tile_name = image_name.split("_",4)[-1].rsplit("_",1)[0] #state-year included in image name
        else:
            tile_name = image_name.rsplit("_",2)[0] #tile name formated image name
        file_names.append(image_name)
        tile_names.append(tile_name)
    tile_names = sorted(np.unique(tile_names))
    file_names = sorted(np.unique(file_names))

    return(file_names, tile_names)

def make_by_tile_dirs(home_dir, tile_name):
    #create folder to store corrected chips/xmls
    tile_dir = os.path.join(home_dir, tile_name) #sub folder for each tile 
    chips_positive_path = os.path.join(tile_dir,"chips_positive") #images path
    chips_positive_xml_path = os.path.join(tile_dir,"chips_positive_xml") #xmls paths
    os.makedirs(tile_dir, exist_ok=True)
    os.makedirs(chips_positive_path, exist_ok=True)
    os.makedirs(chips_positive_xml_path, exist_ok=True)
    return(tile_dir)

def read_tile(tile_path, item_dim = int(512)):
    tile = cv2.imread(tile_path, cv2.IMREAD_UNCHANGED) 
    tile_height,  tile_width,  tile_channels = tile.shape #the size of the tile 
    row_index = math.ceil(tile_height/item_dim) #y
    col_index = math.ceil(tile_width/item_dim) #x
    return(tile, row_index, col_index)

def compare_images(t_2_chip, labeled_img):  
    gray_t_2_chip = cv2.cvtColor(t_2_chip.astype(np.uint8), cv2.COLOR_BGR2GRAY) # make gray
    gray_labeled_image = cv2.cvtColor(labeled_img.astype(np.uint8), cv2.COLOR_BGR2GRAY) #image that has been chipped from tile
    (score, diff) = compare_ssim(gray_t_2_chip, gray_labeled_image, full=True)
    if score >= 0.95: #If the labeled image is correct
        #chip_name_incorrectly_chip_names[index]
        return(True)
    else: #if it is incorrect
        ## move incorrectly named image if it one of the same name has not already been moved
        return(False)

def copy_images_xml(img_name, img_path, xml_path, copy_dir):                  
    ####    
    new_img_path = os.path.join(copy_dir, "chips_positive", img_name + ".jpg")
    if not os.path.exists(new_img_path):
        shutil.copy(img_path, new_img_path)
        
    new_xml_path = os.path.join(copy_dir, "chips_positive_xml", img_name + ".xml")
    if not os.path.exists(new_xml_path):
        shutil.copy(xml_path, new_xml_path) #destination
        
def copy_and_replace_images_xml(img_name, img_path, xml_path, copy_dir):                  
    ####    
    new_img_path = os.path.join(copy_dir, "chips_positive", img_name + ".jpg")
    shutil.copy(img_path, new_img_path)
        
    new_xml_path = os.path.join(copy_dir, "chips_positive_xml", img_name + ".xml")
    shutil.copy(xml_path, new_xml_path) #destination
        
def copy_remaining_images(y, col_index, img, img_name, remaining_img_dir):    
    remaining_img_path = os.path.join(remaining_img_dir, img_name)
    if (y >= col_index): #y: we started at 0 here and 1 before? (save the remaining rows) #x:do not include extraneous black images
        if not os.path.exists(remaining_img_path):
            cv2.imwrite(remaining_img_path, img)   
            
def compile_unverified_dataset(img_paths_anno_paths, tile_dir_path, compiled_by_tile_dir, images_do_not_match_names_dir, correctly_chipped_incorrect_data_dir):
    for directory in tqdm.tqdm(img_paths_anno_paths):
        #get all the image and xml paths in directory of annotated images
        fc.remove_thumbs(directory[0])
        image_paths = sorted(glob(directory[0] + "/*.jpg", recursive = True))
        xml_paths = sorted(glob(directory[1] + "/*.xml", recursive = True))

        #identify tiles in each folder
        file_names, tile_names = get_unique_tile_names(image_paths)
        for tile_name in tile_names:
            compile_tile_dir = make_by_tile_dirs(compiled_by_tile_dir, tile_name)

            img_in_tile_paths = [string for string in image_paths if tile_name in string] 
            xml_in_tile_paths = [string for string in xml_paths if tile_name in string]
            img_in_tile_names = [string for string in file_names if tile_name in string]          
            assert len(img_in_tile_paths) == len(xml_in_tile_paths) == len(img_in_tile_names), "The same number of images and xmls"

            tile, row_index, col_index = read_tile(os.path.join(tile_dir_path, tile_name + ".tif")) #read in tile

            count = 1
            for y in range(0, row_index): #rows
                for x in range(0, col_index): #cols
                    t_2_chip = fc.tile_to_chip_array(tile, x, y, int(512)) #get correct chip from tile

                    #compare image names to standard quad names 
                    standard_quad_img_name_wo_ext = tile_name + '_' + f"{y:02}"  + '_' + f"{x:02}" # row_col
                    standard_quad_index, = np.where(np.array(img_in_tile_names) == standard_quad_img_name_wo_ext) 
                    if len(standard_quad_index) > 0: 
                        index = [standard_quad_index][0][0] # only pull the first, they can only be the same values 
                        img_path = img_in_tile_paths[index]
                        xml_path = xml_in_tile_paths[index]
                        img_name = img_in_tile_names[index]
                        if compare_images(t_2_chip, cv2.imread(img_path)):
                            copy_images_xml(standard_quad_img_name_wo_ext, img_path, xml_path, compile_tile_dir)
                        else: 
                            copy_images_xml(standard_quad_img_name_wo_ext, img_path, xml_path, correctly_chipped_incorrect_data_dir)

                    #compare image names to state year names
                    tile_six_digit_img_name_wo_ext = tile_name + '_'+ str(count).zfill(6) #specify the chip names
                    state_year_img_name_wo_ext = [string for string in img_in_tile_names if tile_six_digit_img_name_wo_ext in string]  
                    if len(state_year_img_name_wo_ext) > 0: 
                        state_year_index, = np.where(np.array(img_in_tile_names) == state_year_img_name_wo_ext[0]) 
                        index = [state_year_index][0][0]
                        img_path = img_in_tile_paths[index]
                        xml_path = xml_in_tile_paths[index]
                        img_name = img_in_tile_names[index]
                        if compare_images(t_2_chip, cv2.imread(img_path)):
                            copy_images_xml(standard_quad_img_name_wo_ext, img_path, xml_path, compile_tile_dir) #use standard name and copy to compiled directory
                        else:
                            copy_images_xml(img_name, img_path, xml_path, images_do_not_match_names_dir) #move to incorrect directory

                    #counter for image pathway
                    count += 1


def compile_verified_standard_dataset(img_paths_anno_paths, tile_dir_path, compiled_by_tile_dir, images_do_not_match_names_dir, correctly_chipped_incorrect_data_dir):
    for directory in tqdm.tqdm(img_paths_anno_paths):
        #get all the image and xml paths in directory of annotated images
        fc.remove_thumbs(directory[0])
        image_paths = sorted(glob(directory[0] + "/*.jpg", recursive = True))
        xml_paths = sorted(glob(directory[1] + "/*.xml", recursive = True))

        #identify tiles in each folder
        file_names, tile_names = get_unique_tile_names(image_paths)
        for tile_name in tile_names:
            compile_tile_dir = make_by_tile_dirs(compiled_by_tile_dir, tile_name)

            img_in_tile_paths = [string for string in image_paths if tile_name in string] 
            xml_in_tile_paths = [string for string in xml_paths if tile_name in string]
            img_in_tile_names = [string for string in file_names if tile_name in string]          
            assert len(img_in_tile_paths) == len(xml_in_tile_paths) == len(img_in_tile_names), "The same number of images and xmls"

            tile, row_index, col_index = read_tile(os.path.join(tile_dir_path, tile_name + ".tif")) #read in tile

            for y in range(0, row_index): #rows
                for x in range(0, col_index): #cols
                    t_2_chip = fc.tile_to_chip_array(tile, x, y, int(512)) #get correct chip from tile

                    #compare image names to standard quad names 
                    standard_quad_img_name_wo_ext = tile_name + '_' + f"{y:02}"  + '_' + f"{x:02}" # row_col
                    standard_quad_index, = np.where(np.array(img_in_tile_names) == standard_quad_img_name_wo_ext) 
                    if len(standard_quad_index) > 0: 
                        index = [standard_quad_index][0][0] # only pull the first, they can only be the same values 
                        img_path = img_in_tile_paths[index]
                        xml_path = xml_in_tile_paths[index]
                        img_name = img_in_tile_names[index]
                        if compare_images(t_2_chip, cv2.imread(img_path)):
                            copy_and_replace_images_xml(standard_quad_img_name_wo_ext, img_path, xml_path, compile_tile_dir)
                        else: 
                            copy_and_replace_images_xml(standard_quad_img_name_wo_ext, img_path, xml_path, correctly_chipped_incorrect_data_dir)
def compile_verified_state_year_dataset(img_paths_anno_paths, tile_dir_path, compiled_by_tile_dir, images_do_not_match_names_dir, correctly_chipped_incorrect_data_dir):
    for directory in tqdm.tqdm(img_paths_anno_paths):
        #get all the image and xml paths in directory of annotated images
        fc.remove_thumbs(directory[0])
        image_paths = sorted(glob(directory[0] + "/*.jpg", recursive = True))
        xml_paths = sorted(glob(directory[1] + "/*.xml", recursive = True))

        #identify tiles in each folder
        file_names, tile_names = get_unique_tile_names(image_paths)
        for tile_name in tile_names:
            compile_tile_dir = make_by_tile_dirs(compiled_by_tile_dir, tile_name)

            img_in_tile_paths = [string for string in image_paths if tile_name in string] 
            xml_in_tile_paths = [string for string in xml_paths if tile_name in string]
            img_in_tile_names = [string for string in file_names if tile_name in string]          
            assert len(img_in_tile_paths) == len(xml_in_tile_paths) == len(img_in_tile_names), "The same number of images and xmls"

            tile, row_index, col_index = read_tile(os.path.join(tile_dir_path, tile_name + ".tif")) #read in tile

            count = 1
            for y in range(0, row_index): #rows
                for x in range(0, col_index): #cols
                    t_2_chip = fc.tile_to_chip_array(tile, x, y, int(512)) #get correct chip from tile
                    #compare image names to state year names
                    standard_quad_img_name_wo_ext = tile_name + '_' + f"{y:02}"  + '_' + f"{x:02}" # row_col
                    tile_six_digit_img_name_wo_ext = tile_name + '_'+ str(count).zfill(6) #specify the chip names
                    state_year_img_name_wo_ext = [string for string in img_in_tile_names if tile_six_digit_img_name_wo_ext in string]  
                    if len(state_year_img_name_wo_ext) > 0: 
                        state_year_index, = np.where(np.array(img_in_tile_names) == state_year_img_name_wo_ext[0]) 
                        index = [state_year_index][0][0]
                        img_path = img_in_tile_paths[index]
                        xml_path = xml_in_tile_paths[index]
                        img_name = img_in_tile_names[index]
                        if compare_images(t_2_chip, cv2.imread(img_path)):
                            copy_and_replace_images_xml(standard_quad_img_name_wo_ext, img_path, xml_path, compile_tile_dir) #use standard name and copy to compiled directory
                        else:
                            copy_and_replace_images_xml(img_name, img_path, xml_path, images_do_not_match_names_dir) #move to incorrect directory
                    #counter for image pathway
                    count += 1
def compile_verified_state_year_dataset(img_paths_anno_paths, tile_dir_path, compiled_by_tile_dir, images_do_not_match_names_dir, correctly_chipped_incorrect_data_dir):
    img_count = 0
    image_paths = []
    xml_paths = []
    for directory in all_verified_state_year_subfolders_path:
        #get all the image and xml paths in directory of annotated images
        fc.remove_thumbs(directory[0])
        image_paths += sorted(glob(directory[0] + "/*.jpg", recursive = True))
        xml_paths += sorted(glob(directory[1] + "/*.xml", recursive = True))
    #identify tiles in each folder
    file_names, tile_names = get_unique_tile_names(image_paths)
        
    for tile_name in tqdm.tqdm(tile_names):
        compile_tile_dir = make_by_tile_dirs(compiled_by_tile_dir, tile_name)

        img_in_tile_paths = [string for string in image_paths if tile_name in string] 
        xml_in_tile_paths = [string for string in xml_paths if tile_name in string]
        img_in_tile_names = [string for string in file_names if tile_name in string]          
        assert len(img_in_tile_paths) == len(xml_in_tile_paths) == len(img_in_tile_names), "The same number of images and xmls"

        tile, row_index, col_index = read_tile(os.path.join(tile_dir_path, tile_name + ".tif")) #read in tile

        count = 1
        for y in range(0, row_index): #rows
            for x in range(0, col_index): #cols
                t_2_chip = fc.tile_to_chip_array(tile, x, y, int(512)) #get correct chip from tile
                #compare image names to state year names
                standard_quad_img_name_wo_ext = tile_name + '_' + f"{y:02}"  + '_' + f"{x:02}" # row_col
                tile_six_digit_img_name_wo_ext = tile_name + '_'+ str(count).zfill(6) #specify the chip names
                state_year_img_name_wo_ext = [string for string in img_in_tile_names if tile_six_digit_img_name_wo_ext in string]
                state_year_index, = np.where(np.array(img_in_tile_names) == state_year_img_name_wo_ext) 

                if len(state_year_index) >= 1: 
                    #print(state_year_index[0])
                    index = state_year_index[0]
                    img_path = img_in_tile_paths[index]
                    xml_path = xml_in_tile_paths[index]
                    img_name = img_in_tile_names[index]
                    if compare_images(t_2_chip, cv2.imread(img_path)):
                        img_count += 1
                        copy_and_replace_images_xml(standard_quad_img_name_wo_ext, img_path, xml_path, compile_tile_dir) #use standard name and copy to compiled directory
                    else:
                        copy_and_replace_images_xml(img_name, img_path, xml_path, images_do_not_match_names_dir) #move to incorrect directory
                #counter for image pathway
                count += 1
    print(img_count)


In [4]:
verified_state_year_subfolders_path = os.path.join(parent_directory,"verified/verified_state_year_image_name_formating")
all_verified_state_year_subfolders_path = ap.img_path_anno_path(ap.list_of_sub_directories(verified_state_year_subfolders_path)) 

In [49]:
def get_tile_names(paths):
    tile_names = []
    img_names = []
    for path in paths:
        img_name = os.path.splitext(os.path.basename(path))[0]
        if img_name.count("_") > 9:
            tile_name = img_name.split("_",4)[-1].rsplit("_",1)[0] #state-year included in image name
        else:
            tile_name = img_name.rsplit("_",2)[0] #tile name formated image name
        img_names.append(img_name)
        tile_names.append(tile_name)
    tile_names = sorted(tile_names)
    img_names = sorted(img_names)
    return(tile_names, img_names)

#compare image names to state year names
def copy_and_replace_images_xml(img_name, img_path, xml_path, copy_dir):                  
    ####    
    new_img_path = os.path.join(copy_dir, "chips_positive", img_name + ".jpg")
    shutil.copy(img_path, new_img_path)
        
    new_xml_path = os.path.join(copy_dir, "chips_positive_xml", img_name + ".xml")
    shutil.copy(xml_path, new_xml_path) #destination
def compare_move_images_state_years(x, y, tile_name, img_in_tile_names, count, count_dif, img_count):
    standard_quad_img_name_wo_ext = tile_name + '_' + f"{y:02}"  + '_' + f"{x:02}" # row_col
    img_name_wo_ext = tile_name + '_'+ str(count + count_dif).zfill(6) #specify the chip names
    state_year_img_name_wo_ext = [string for string in img_in_tile_names if img_name_wo_ext in string]

    assert len(set(state_year_img_name_wo_ext)) <= 1, "to many names"
    state_year_img_name_wo_ext = list(set(state_year_img_name_wo_ext))
    state_year_index, = np.where(np.array(img_in_tile_names) == state_year_img_name_wo_ext) 
    
    if len(state_year_index) >= 1: 
        for index in state_year_index:

            img_path = img_in_tile_paths[index]
            xml_path = xml_in_tile_paths[index]
            img_name = img_in_tile_names[index]
            
            if compare_images(t_2_chip, cv2.imread(img_path)):
                print(img_name_wo_ext, state_year_img_name_wo_ext)
                print(x,y, count_dif, img_path)
                img_count += 1
                copy_and_replace_images_xml(standard_quad_img_name_wo_ext, img_path, xml_path, compile_tile_dir) #use standard name and copy to compiled directory
            else:
                copy_and_replace_images_xml(img_name, img_path, xml_path, images_do_not_match_names_dir) #move to incorrect directory
    return(img_count)
        #counter for image pathway

In [50]:
img_count = 0
for tile_name in tqdm.tqdm([tile_names[0]]):
    compile_tile_dir = make_by_tile_dirs(compiled_by_tile_dir, tile_name)

    img_in_tile_paths = [string for string in image_paths if tile_name in string] 
    xml_in_tile_paths = [string for string in xml_paths if tile_name in string]
    img_in_tile_names = [string for string in img_names if tile_name in string]          
    assert len(img_in_tile_paths) == len(xml_in_tile_paths) == len(img_in_tile_names), "The same number of images and xmls"
    
    tile, row_index, col_index = read_tile(os.path.join(tile_dir_path, tile_name + ".tif")) #read in tile
    print(row_index, col_index)
    print(img_in_tile_paths)
    count = 0
    for y in range(0, row_index): #rows
        for x in range(0, col_index): #cols
            t_2_chip = fc.tile_to_chip_array(tile, x, y, int(512)) #get correct chip from tile
            img_count = compare__move_images_state_years(x, y, tile_name, img_in_tile_names, count, -2, img_count)
            img_count = compare__move_images_state_years(x, y, tile_name, img_in_tile_names, count, -1, img_count)
            img_count = compare__move_images_state_years(x, y, tile_name, img_in_tile_names, count, 0, img_count)
            img_count = compare__move_images_state_years(x, y, tile_name, img_in_tile_names, count, 1, img_count)
            img_count = compare__move_images_state_years(x, y, tile_name, img_in_tile_names, count, 2, img_count)

            count += 1
    print(img_count)

  0%|          | 0/1 [00:00<?, ?it/s]

24 23
['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//verified/verified_state_year_image_name_formating/verify_1_josh_jaewon_amadu/chips_positive\\fl_60cm_2019_24081_m_2408126_ne_17_060_20191128_000044.jpg', '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//verified/verified_state_year_image_name_formating/verify_1_josh_jaewon_amadu/chips_positive\\fl_60cm_2019_24081_m_2408126_ne_17_060_20191128_000319.jpg', '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//verified/verified_state_year_image_name_formating/verify_1_josh_jaewon_amadu/chips_positive\\fl_60cm_2019_24081_m_2408126_ne_17_060_20191128_000439.jpg', '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//verified/verified_state_year_image_name_formating/verify_1_josh_jaewon_amadu/chips_positive\\fl_60cm_2019_24081_m_2408126_ne_17_060_20191128_000454.jpg', '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//verified/verified_state_year_image_name_formating/verify_1_josh_jaewon_amadu/chips_positive\\fl_60cm_2019_24081_

C:\Users\RAPIDU~1\AppData\Local\Temp/ipykernel_7456/725189633.py:24: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  state_year_index, = np.where(np.array(img_in_tile_names) == state_year_img_name_wo_ext)
100%|██████████| 1/1 [00:21<00:00, 21.51s/it]

1


In [19]:
            #compare image names to state year names
            standard_quad_img_name_wo_ext = tile_name + '_' + f"{y:02}"  + '_' + f"{x:02}" # row_col
            
            six_digit_zero_count_img_name_wo_ext = tile_name + '_'+ str(count).zfill(6) #specify the chip names
            state_year_zero_count_img_name_wo_ext = [string for string in img_in_tile_names if six_digit_zero_count_img_name_wo_ext in string]
            
            assert len(set(state_year_zero_count_img_name_wo_ext)) <= 1, "to many names"
            state_year_zero_count_img_name_wo_ext = list(set(state_year_zero_count_img_name_wo_ext))
            state_year_zero_count_index, = np.where(np.array(img_in_tile_names) == state_year_zero_count_img_name_wo_ext) 


            if len(state_year_index) >= 1: 
                for index in state_year_index:
                    img_path = img_in_tile_paths[index]
                    xml_path = xml_in_tile_paths[index]
                    img_name = img_in_tile_names[index]
                    if compare_images(t_2_chip, cv2.imread(img_path)):
                        img_count += 1
                        copy_and_replace_images_xml(standard_quad_img_name_wo_ext, img_path, xml_path, compile_tile_dir) #use standard name and copy to compiled directory
                    else:
                        copy_and_replace_images_xml(img_name, img_path, xml_path, images_do_not_match_names_dir) #move to incorrect directory
                #counter for image pathway
            
            six_digit_one_count_img_name_wo_ext = tile_name + '_'+ str(count + 1).zfill(6) #specify the chip names
            state_year_one_count_img_name_wo_ext = [string for string in img_in_tile_names if six_digit_one_count_img_name_wo_ext in string]
            
            assert len(set(state_year_one_count_img_name_wo_ext)) <= 1, "to many names"
            state_year_one_count_img_name_wo_ext = list(set(state_year_one_count_img_name_wo_ext))
            state_year_one_count_index, = np.where(np.array(img_in_tile_names) == state_year_one_count_img_name_wo_ext) 

            if len(state_year_index) >= 1: 
                for index in state_year_index:
                    img_path = img_in_tile_paths[index]
                    xml_path = xml_in_tile_paths[index]
                    img_name = img_in_tile_names[index]
                    if compare_images(t_2_chip, cv2.imread(img_path)):
                        img_count += 1
                        copy_and_replace_images_xml(standard_quad_img_name_wo_ext, img_path, xml_path, compile_tile_dir) #use standard name and copy to compiled directory
                    else:
                        copy_and_replace_images_xml(img_name, img_path, xml_path, images_do_not_match_names_dir) #move to incorrect directory
                #counter for image pathway

array([], dtype=int64)

In [ ]:
unverified_subfolders_path = os.path.join(parent_directory,"unverified_images")

all_unverified_subfolders_path = []
for dir_ in (ap.list_of_sub_directories(unverified_subfolders_path)):
    img_path_by_unverified_set = ap.img_path_anno_path(ap.list_of_sub_directories(dir_))
    for paths in img_path_by_unverified_set:
        all_unverified_subfolders_path.append(paths)
        
compile_unverified_dataset(all_unverified_subfolders_path[18:], tile_dir_path, compiled_by_tile_dir, images_do_not_match_names_dir, correctly_chipped_incorrect_data_dir)

In [111]:
verified_state_year_subfolders_path = os.path.join(parent_directory,"verified/verified_state_year_image_name_formating")
all_verified_state_year_subfolders_path = ap.img_path_anno_path(ap.list_of_sub_directories(verified_state_year_subfolders_path)) 

compile_verified_state_year_dataset(all_verified_state_year_subfolders_path, tile_dir_path, compiled_by_tile_dir, 
                 images_do_not_match_names_dir, correctly_chipped_incorrect_data_dir)

  0%|          | 0/2139 [00:00<?, ?it/s]C:\Users\RAPIDU~1\AppData\Local\Temp/ipykernel_10100/3046656880.py:228: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  state_year_index, = np.where(np.array(img_in_tile_names) == state_year_img_name_wo_ext)
  1%|          | 16/2139 [02:28<5:27:52,  9.27s/it]


AssertionError: The same number of images and xmls

In [22]:
verified_standard_quad_subfolders_path = os.path.join(parent_directory,"verified/verified_standard_quad_image_name_format")

all_verified_standard_quad_subfolders_path =  ap.img_path_anno_path(ap.list_of_sub_directories(verified_standard_quad_subfolders_path))
compile_verified_standard_dataset(all_verified_standard_quad_subfolders_path, tile_dir_path, compiled_by_tile_dir, images_do_not_match_names_dir, correctly_chipped_incorrect_data_dir)

100%|██████████| 7/7 [2:56:14<00:00, 1510.70s/it]  


In [ ]:
#create renamed xml for all xmls in dataset
reformat_xmls_for_rechipped_images(xml_directory, image_in_tile, correct_xml_name, correct_jpg_name, chips_positive_xml_dir_path)

In [64]:
tile_paths = glob(tile_dir_path + "/*.tif", recursive = True)
tile_names, _ = get_unique_tile_names(tile_paths)

incorrect_img_paths = sorted(glob(os.path.join(images_do_not_match_names_dir,"chips_positive") + "/*.jpg", recursive = True))
incorrect_img_names = []
for img_path in incorrect_img_paths:
    img_name = os.path.splitext(os.path.basename(img_path))[0]
    incorrect_img_names.append(img_name)
    
for tile_name in tqdm.tqdm(tile_names):
    tile, row_index, col_index = read_tile(os.path.join(tile_dir_path, tile_name + ".tif")) #read in tile
    count = 1
    for y in range(0, row_index): #rows
        for x in range(0, col_index): #cols
            t_2_chip = fc.tile_to_chip_array(tile, x, y, int(512)) #get correct chip from tile
            
            incorrect_img_names_copy = copy.copy(incorrect_img_names)
            for img_name in incorrect_img_names_copy:
                img_path = os.path.join(images_do_not_match_names_dir,"chips_positive", img_name+".jpg")
                xml_path = os.path.join(images_do_not_match_names_dir,"chips_positive_xml", img_name+".xml")
                if compare_images(t_2_chip, cv2.imread(img_path)):
                    
                    standard_quad_img_name_wo_ext = tile_name + '_' + f"{y:02}"  + '_' + f"{x:02}" # row_col
                    copy_and_replace_images_xml(standard_quad_img_name_wo_ext, img_path, xml_path, 
                                                os.path.join(images_do_not_match_names_dir,"chips_positive"))
                    print(img_name, standard_quad_img_name_wo_ext)
                    incorrect_img_names_copy.remove(img_name)
                    
            incorrect_img_paths = copy.copy(incorrect_img_names_copy)
            count += 1

  0%|          | 0/2154 [1:04:10<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
count = 1
for y in range(0, col_index): #rows
    for x in range(0, col_index): #cols
        t_2_chip = fc.tile_to_chip_array(tile, x, y, int(512)) #get correct chip from tile
        #compare image names to state year names
        standard_quad_img_name_wo_ext = tile_name + '_' + f"{y:02}"  + '_' + f"{x:02}" # row_col
        tile_six_digit_img_name_wo_ext = tile_name + '_'+ str(count).zfill(6) #specify the chip names
        state_year_img_name_wo_ext = [string for string in img_in_tile_names if tile_six_digit_img_name_wo_ext in string]  
        if len(state_year_img_name_wo_ext) > 0: 
            state_year_index, = np.where(np.array(img_in_tile_names) == state_year_img_name_wo_ext[0]) 
            index = [state_year_index][0][0]
            img_path = img_in_tile_paths[index]
            xml_path = xml_in_tile_paths[index]
            img_name = img_in_tile_names[index]
            if compare_images(t_2_chip, cv2.imread(img_path)):
                copy_and_replace_images_xml(standard_quad_img_name_wo_ext, img_path, xml_path, compile_tile_dir) #use standard name and copy to compiled directory
            else:
                copy_and_replace_images_xml(img_name, img_path, xml_path, images_do_not_match_names_dir) #move to incorrect directory
        #counter for image pathway
        count += 1

In [ ]:
def img_anno_paths_to_corrected_names_for_labeled_images_and_remaining_images(img_paths_anno_paths, correct_directory, incorrectly_chipped_images_path, 
                                                                              remaining_chips_path, tiles_complete_dataset_path):
    """ iterate over all the image and xml paths in directory of annotated images; identify tiles and the corresonding images/xmls in each folder;
    match name of previous naming convention and row/col naming convention;for labeled images and xmls, create folder to store, 
    identify correct images, copy, and rename; identify and save remaining images; 
    Args: 
    img_paths_anno_paths(np array): n x 2 array of jpg and xml paths
    correct_directory(str): path to directory containing xmls and images with correct names
    incorrectly_chipped_images_path(str): path to folder that will contain all of the incorrect named, images chipped from times
    remaining_chips_path(str): path to folder that will contain all of the remaining images that have not been labeled and correspond to tiles that have labeled images
    tiles_complete_dataset_path(str): path to folder containing tiles
    
    Returns:
    multiple_annotations_images: list of images with multiple annotations in a given folder
    black_images_with_annotations: list of black images with annotations
    """
    multiple_annotations_images = []
    black_images_with_annotations = []
    for directory in tqdm.tqdm(img_paths_anno_paths):
        #get all the image and xml paths in directory of annotated images
        print(directory)
        remove_thumbs(directory[0])
        image_paths = sorted(glob(directory[0] + "/*.jpg", recursive = True))
        xml_paths = sorted(glob(directory[1] + "/*.xml", recursive = True))
        #print(len(image_paths),len(xml_paths))

        #identify tiles in each folder
        tiles = []
        for image in image_paths:
            image_name = os.path.splitext(os.path.basename(image))[0]
            if image_name.count("_") > 9:
                tile_name = image_name.split("_",4)[-1].rsplit("_",1)[0] #state included in image name
            else:
                tile_name = image_name.rsplit("_",2)[0] #tile name formated image name
            
            tiles.append(tile_name)
        tiles = sorted(np.unique(tiles))

        #identify the images/xmls that correspond with each tile in folder
        for tile_name in tiles:        
            images_in_tile = [string for string in image_paths if tile_name in string]          
            xmls_in_tile = [string for string in xml_paths if tile_name in string]  
            assert len(images_in_tile) == len(xmls_in_tile), "The same number of images and xmls"
            #print(tile_name, len(images_in_tile))

            #create folder to store corrected chips/xmls
            tile_dir_path = os.path.join(correct_directory, tile_name) #sub folder for each tile 
            chips_positive_dir_path = os.path.join(tile_dir_path,"chips_positive") #images path
            chips_positive_xml_dir_path = os.path.join(tile_dir_path,"chips_positive_xml") #xmls paths

            tile_dir = os.makedirs(tile_dir_path, exist_ok=True)
            chips_positive_dir = os.makedirs(chips_positive_dir_path, exist_ok=True)
            chips_positive_xml_dir = os.makedirs(chips_positive_xml_dir_path, exist_ok=True)

            #identify and save remaining images; match name of previous naming convention and row/col naming convention
            ys, xs, chip_name_incorrectly_chip_names, chip_name_correct_chip_names =  incorrectly_chipped_image_and_correctly_chipped_names(incorrectly_chipped_images_path, remaining_chips_path,                                                                                        tiles_complete_dataset_path, tile_name)

            #identify labeled images that are correct; copy and rename correct images and xmls
            multiple_annotations_images, black_images_with_annotations = copy_rename_labeled_images_xmls(directory[1], images_in_tile, incorrectly_chipped_images_path,
                                                                                                            chips_positive_dir_path, chips_positive_xml_dir_path,
                                                                                                            chip_name_incorrectly_chip_names, chip_name_correct_chip_names,
                                                                                                            multiple_annotations_images, black_images_with_annotations)
        #remaining images
        print("remaining images", len(os.listdir(remaining_chips_path)))
        

# End Testing

In [3]:
unverified_set1_subfolders_path = os.path.join(parent_directory,"unverified_images\student_reviewed_unverified_images_set1")
unverified_set1_subfolders_paths = ap.img_path_anno_path(ap.list_of_sub_directories(unverified_set1_subfolders_path))
fc.img_anno_paths_to_corrected_names_for_labeled_images_and_remaining_images(unverified_set1_subfolders_paths[32:], correct_directory, incorrectly_chipped_images_path, 
                                                                              remaining_chips_path, tiles_complete_dataset_path)

  0%|          | 0/8 [00:00<?, ?it/s]

['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set1/Qianyu/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set1/Qianyu/chips_positive_xml']
24 22
remaining images 62692
24 22
remaining images 62692
25 22
remaining images 62692
25 22
remaining images 62692
25 22
remaining images 62692
25 22
remaining images 62692
25 22
remaining images 62692
25 23
remaining images 62692
26 23
remaining images 62692
26 23
remaining images 62692
25 23
remaining images 62692
25 23
remaining images 62692
26 23
remaining images 62692
25 23
remaining images 62692
25 23
remaining images 62692
26 23
remaining images 62692
25 23
remaining images 62692
26 23
remaining images 62692
26 23
remaining images 62692
26 23
remaining images 62692
26 23
remaining images 62692
25 23
remaining images 62692
26 23
remaining images 62692
26 23
remaining images 62692
26 23
remaining images 6269

 12%|█▎        | 1/8 [24:43<2:53:05, 1483.59s/it]

['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set1/Sunny/student_reviewed_images1/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set1/Sunny/student_reviewed_images1/chips_positive_xml']
25 17
remaining images 62928
25 17
remaining images 63064
25 17
remaining images 63200
25 17
remaining images 63200
25 18
remaining images 63200
25 17
remaining images 63336
25 18
remaining images 63336
25 17
remaining images 63472
25 17
remaining images 63608
25 17
remaining images 63744
25 17
remaining images 63880
25 18
remaining images 64006
25 18
remaining images 64132
25 18
remaining images 64258
25 18
remaining images 64384
25 17
remaining images 64520
25 17
remaining images 64656
25 17
remaining images 64792
25 17
remaining images 64928
25 17
remaining images 65064
25 18
remaining images 65190
25 18
remaining images 65316
25 18
remaining images 65442
25 18
re

 25%|██▌       | 2/8 [1:03:54<3:19:24, 1994.05s/it]

...

 75%|███████▌  | 6/8 [2:45:38<46:43, 1401.77s/it]  

['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set1/Sunny/student_reviewed_images6/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set1/Sunny/student_reviewed_images6/chips_positive_xml']
25 20
remaining images 81868
25 20
remaining images 81868
25 20
remaining images 81968
25 20
remaining images 81968
25 19
remaining images 81968
25 19
remaining images 82082
25 19
remaining images 82196
25 19
remaining images 82196
25 19
remaining images 82196
25 19
remaining images 82310
25 19
remaining images 82310
24 19
remaining images 82310
24 19
remaining images 82310
24 19
remaining images 82310
24 19
remaining images 82310
24 19
remaining images 82310
24 19
remaining images 82310
24 19
remaining images 82405
24 19
remaining images 82500
25 19
remaining images 82614
25 20
remaining images 82714
25 20
remaining images 82814
25 20
remaining images 82914
25 20
re

 88%|████████▊ | 7/8 [2:59:01<20:06, 1206.13s/it]

['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set1/Sunny/student_reviewed_images7/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set1/Sunny/student_reviewed_images7/chips_positive_xml']
26 20
remaining images 83134
25 20
remaining images 83234
25 20
remaining images 83334
25 21
remaining images 83418
26 20
remaining images 83538
25 20
remaining images 83638
26 21
remaining images 83743
26 21
remaining images 83848
25 20
remaining images 83948
25 20
remaining images 83948


100%|██████████| 8/8 [3:05:39<00:00, 1392.48s/it]


### Set 2

In [4]:
unverified_set2_subfolders_path = os.path.join(parent_directory,"unverified_images\student_reviewed_unverified_images_set2")
unverified_set2_subfolders_paths = ap.img_path_anno_path(ap.list_of_sub_directories(unverified_set2_subfolders_path))

fc.img_anno_paths_to_corrected_names_for_labeled_images_and_remaining_images(unverified_set2_subfolders_paths[45:], correct_directory, incorrectly_chipped_images_path, 
                                                                              remaining_chips_path, tiles_complete_dataset_path)

  0%|          | 0/11 [00:00<?, ?it/s]

['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set2/Sunny/student_reviewed_images12_Sunny/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set2/Sunny/student_reviewed_images12_Sunny/chips_positive_xml']


  9%|▉         | 1/11 [04:09<41:30, 249.09s/it]

remaining images 133271
['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set2/Sunny/student_reviewed_images13_Sunny/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set2/Sunny/student_reviewed_images13_Sunny/chips_positive_xml']


 18%|█▊        | 2/11 [10:56<51:18, 342.05s/it]

...

remaining images 141837
['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set2/Tang/student_reviewed_images5_Tang/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set2/Tang/student_reviewed_images5_Tang/chips_positive_xml']


 91%|█████████ | 10/11 [57:54<05:47, 347.61s/it]

remaining images 142804
['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set2/Tang/student_reviewed_images6_Tang/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set2/Tang/student_reviewed_images6_Tang/chips_positive_xml']


100%|██████████| 11/11 [1:03:15<00:00, 345.01s/it]

remaining images 143605


### Set 3

In [36]:
unverified_set3_subfolders_path = os.path.join(parent_directory,"unverified_images\student_reviewed_unverified_images_set3")
unverified_set3_subfolders_paths = ap.img_path_anno_path(ap.list_of_sub_directories(unverified_set3_subfolders_path))
fc.img_anno_paths_to_corrected_names_for_labeled_images_and_remaining_images(unverified_set3_subfolders_paths, correct_directory, incorrectly_chipped_images_path, 
                                                                              remaining_chips_path, tiles_complete_dataset_path)

  0%|          | 0/31 [00:00<?, ?it/s]

['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set3/Alvarez/student_reviewed_images4_Alvarez/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set3/Alvarez/student_reviewed_images4_Alvarez/chips_positive_xml']


  3%|▎         | 1/31 [08:06<4:03:24, 486.80s/it]

remaining images 144679
['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set3/Alvarez/unverified_images_not_reviewed_by_student5_Alvarez/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set3/Alvarez/unverified_images_not_reviewed_by_student5_Alvarez/chips_positive_xml']


  6%|▋         | 2/31 [14:18<3:22:25, 418.81s/it]

...

remaining images 180235
['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set3/Tang/student_reviewed_images7_Tang/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set3/Tang/student_reviewed_images7_Tang/chips_positive_xml']


 97%|█████████▋| 30/31 [3:23:30<05:06, 306.52s/it]

remaining images 181082
['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set3/Tang/student_reviewed_images8_Tang/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set3/Tang/student_reviewed_images8_Tang/chips_positive_xml']


100%|██████████| 31/31 [3:28:27<00:00, 403.46s/it]

remaining images 181987


### Set 4

In [43]:
unverified_set4_subfolders_path = os.path.join(parent_directory,"unverified_images\student_reviewed_unverified_images_set4")
unverified_set4_subfolders_paths = ap.img_path_anno_path(ap.list_of_sub_directories(unverified_set4_subfolders_path))

fc.img_anno_paths_to_corrected_names_for_labeled_images_and_remaining_images(unverified_set4_subfolders_paths, correct_directory, incorrectly_chipped_images_path, 
                                                                              remaining_chips_path, tiles_complete_dataset_path)

  0%|          | 0/14 [00:00<?, ?it/s]

['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set4/Feinberg/student_reviewed_images7_Feinberg/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set4/Feinberg/student_reviewed_images7_Feinberg/chips_positive_xml']


  7%|▋         | 1/14 [10:27<2:15:52, 627.11s/it]

remaining images 183267
['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set4/Feinberg/student_reviewed_images8_Feinberg/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set4/Feinberg/student_reviewed_images8_Feinberg/chips_positive_xml']


 14%|█▍        | 2/14 [24:29<2:30:41, 753.47s/it]

...

remaining images 192403
['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set4/Poonacha/unverified_images_not_reviewed_by_student15_Poonacha/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set4/Poonacha/unverified_images_not_reviewed_by_student15_Poonacha/chips_positive_xml']


 93%|█████████▎| 13/14 [1:24:40<05:52, 352.38s/it]

remaining images 193292
['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set4/Poonacha/unverified_images_not_reviewed_by_student16_Poonacha/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set4/Poonacha/unverified_images_not_reviewed_by_student16_Poonacha/chips_positive_xml']


100%|██████████| 14/14 [1:28:37<00:00, 379.80s/it]

remaining images 193991


## Verified Sets 

In [116]:
verified_set_path = os.path.join(parent_directory,"verified/verified_state_year_image_name_format")
verified_set_paths = ap.img_path_anno_path(ap.list_of_sub_directories(verified_set_path))
fc.img_anno_paths_to_corrected_names_for_labeled_images_and_remaining_images(verified_set_paths, correct_directory, incorrectly_chipped_images_path, 
                                                                              remaining_chips_path, tiles_complete_dataset_path)

  0%|                                                                                           | 0/23 [00:00<?, ?it/s]

['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//verified/verification_set6/verify_Cleave_Sunny_Robinson_6/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//verified/verification_set6/verify_Cleave_Sunny_Robinson_6/chips_positive_xml']


  0%|                                                                                           | 0/23 [00:13<?, ?it/s]


KeyboardInterrupt: 

# Images with standard quadrangle naming format
- Confirm that the name correctly matches the content (the name and tile/position match)

In [ ]:
# add chips to rechipped folder
for tile_name in tqdm.tqdm(folders_of_images_xmls_by_tile):
    add_chips_to_chip_folders(img_anno_directory, tile_name)

In [73]:
fc.standard_name_verified_images_to_img_anno_by_tile_dir(verified_set_paths, img_anno_directory, incorrect_named_correctly_chipped_dir, tile_dir_path)

100%|████████████████████████████████████████████████████████████████████████████████| 7/7 [2:50:22<00:00, 1460.35s/it]


In [ ]:
fc.relocate_incorrect_image_content_and_naming(img_anno_directory, incorrect_named_correctly_chipped_dir, tile_dir_path)

In [5]:
(glob(incorrect_named_correctly_chipped_dir + "/*.jpg", recursive = True))

[]

In [4]:
#fc.identify_correct_name_incorrectly_named_chipped_images(img_anno_directory, incorrect_named_correctly_chipped_dir, tile_dir_path)

  2%|█▊                                                                             | 28/1184 [00:00<00:04, 275.83it/s]

m_2408126_ne_17_060_20191128
m_2508015_se_17_060_20191120
m_2708203_sw_17_060_20191111
m_2708220_se_17_060_20191129
m_2709841_se_14_060_20180526
m_2709847_se_14_060_20181209
m_2709912_ne_14_060_20190104
m_2709921_sw_14_060_20190104
m_2709929_sw_14_060_20190104
m_2808138_sw_17_060_20191130
m_2809726_sw_14_060_20181209
m_2809839_se_14_060_20181103
m_2908901_ne_16_060_20190707
m_2908901_nw_16_060_20190707
m_2908917_nw_16_060_20190707
m_2908946_sw_16_060_20190906
m_2909006_ne_15_060_20190707
m_2909006_nw_15_060_20190707
m_2909008_se_15_060_20190707
m_2909008_sw_15_060_20190707
m_2909016_ne_15_060_20190707
m_2909019_sw_15_060_20190707
m_2909027_ne_15_060_20190906
m_2909027_nw_15_060_20190906
m_2909038_se_15_060_20190707
m_2909055_sw_15_060_20190830
m_2909121_nw_15_060_20190706
m_2909122_nw_15_060_20190707
m_2909123_nw_15_060_20190707
m_2909123_sw_15_060_20190707
m_2909208_sw_15_060_20190706
m_2909309_se_15_060_20190726
m_2909310_sw_15_060_20190726
m_2909406_se_15_060_20181204
m_2909409_ne_1

  7%|█████▋                                                                         | 85/1184 [00:00<00:04, 249.30it/s]

m_2909515_sw_15_060_20201130
m_2909516_sw_15_060_20201118
m_2909521_se_15_060_20181204
m_2909522_ne_15_060_20201130
m_2909523_ne_15_060_20201130
m_2909523_nw_15_060_20201130
m_2909524_ne_15_060_20201118
m_2909524_nw_15_060_20201118
m_2909524_se_15_060_20181120
m_2909524_se_15_060_20201118
m_2909524_sw_15_060_20201118
m_2909532_ne_15_060_20181120
m_2909532_sw_15_060_20181210
m_2909558_ne_15_060_20181215
m_2909559_ne_15_060_20181210
m_2909559_nw_15_060_20181210
m_2909601_ne_14_060_20181116
m_2909636_nw_14_060_20181201
m_2909643_ne_14_060_20181201
m_2909742_se_14_060_20181101
m_2909829_sw_14_060_20181102
m_2909836_ne_14_060_20181102
m_2909837_ne_14_060_20181102
m_2909837_nw_14_060_20181102
m_3008135_se_17_060_20191119
m_3008136_sw_17_060_20191119
m_3008143_ne_17_060_20191119
m_3008251_ne_17_060_20191110
m_3008816_se_16_060_20191115
m_3008816_sw_16_060_20191115
m_3008824_ne_16_060_20191115
m_3008837_sw_16_060_20181204
m_3009049_nw_15_060_20190906
m_3009058_sw_15_060_20190831
m_3009059_ne_1

 12%|█████████▍                                                                    | 143/1184 [00:00<00:04, 259.78it/s]

...

 95%|████████████████████████████████████████████████████████████████████████▊    | 1120/1184 [00:04<00:00, 265.67it/s]

m_4406810_se_19_060_20180720
m_4406826_ne_19_060_20180720
m_4407339_nw_18_060_20180914
m_4408511_se_16_060_20180912
m_4408523_nw_16_060_20180912
m_4408832_se_16_060_20180911
m_4408928_sw_16_060_20180910
m_4409105_se_15_060_20180908
m_4409107_sw_15_060_20181016
m_4409209_nw_15_060_20190731
m_4409261_sw_15_060_20190820
m_4409262_se_15_060_20190820
m_4409308_se_15_060_20190731
m_4409314_sw_15_060_20190727
m_4409316_ne_15_060_20190731
m_4409316_se_15_060_20190731
m_4409323_ne_15_060_20190727
m_4409456_sw_15_060_20190806
m_4409463_se_15_060_20190806
m_4409534_ne_15_060_20190808
m_4409534_nw_15_060_20190808
m_4409708_sw_14_060_20181015
m_4409837_sw_14_060_20180907
m_4509543_ne_15_060_20190808
m_4509543_se_15_060_20190808
m_4509828_se_14_060_20180917
m_4512219_sw_10_060_20190826
m_4512227_nw_10_060_20190826
m_4606750_se_19_060_20180720
m_4609216_sw_15_060_20190928
m_4609221_se_15_060_20190731
m_4609224_ne_15_060_20181022
m_4609224_ne_15_060_20190928
m_4609224_nw_15_060_20181022
m_4609224_nw_1

100%|████████████████████████████████████████████████████████████████████████████▊| 1181/1184 [00:04<00:00, 275.72it/s]

m_4709624_nw_14_060_20190723
m_4709707_ne_14_060_20190714
m_4709708_nw_14_060_20180901
m_4709708_nw_14_060_20190714
m_4709708_nw_14_060_20190723
m_4709833_ne_14_060_20180723
m_4709833_ne_14_060_20190727
m_4710312_se_13_060_20190718
m_4710322_ne_13_060_20190718
m_4710414_se_13_060_20190713
m_4710458_ne_13_060_20190713
m_4711131_sw_12_060_20190727
m_4711722_nw_11_060_20190726
m_4711722_sw_11_060_20190726
m_4711954_ne_11_060_20190730
m_4711954_se_11_060_20190730
m_4712230_nw_10_060_20191011
m_4712238_se_10_060_20191011
m_4712239_nw_10_060_20191011
m_4712245_se_10_060_20191011
m_4809707_nw_14_060_20180901
m_4809707_nw_14_060_20190707
m_4809707_nw_14_060_20190723
m_4810123_nw_14_060_20181020
m_4810123_nw_14_060_20190714
m_4810131_sw_14_060_20181020
m_4810131_sw_14_060_20190714
m_4810146_se_14_060_20181020
m_4810146_se_14_060_20190714
m_4810204_se_13_060_20180815
m_4810204_se_13_060_20190720
m_4810233_se_13_060_20180815
m_4810233_se_13_060_20190705
m_4810241_se_13_060_20180815
m_4810241_se_1

100%|█████████████████████████████████████████████████████████████████████████████| 1184/1184 [00:04<00:00, 252.79it/s]
